In [3]:
import gzip
import re
import lemmadict

In [25]:
def read_data(file_to_read, rows):
    d = {}
    root = '#'
    list_of_POS = ['ADV','ADJ','PRON','NOUN','VERB','PROPN','CONJ','NUM','SCONJ','ADP', 'INTJ']
    error_count = 0
    uusia = 0
    olevia = 0
    with gzip.open(file_to_read, 'r') as rf:
        
        for i, line in enumerate(rf):
            if re.match('#', line) or len(line) <= 1:
                continue
            elif line.strip() in list_of_POS:
                root = line.strip()
            else:
                #Just wanted adjectives nothing else
                if(root != 'ADJ'):
                    continue
                #split csv lines to list
                cols = line.split(',')
                #everything in lowercase and extra padding away spaces tabs etc. and utf8
                cols = [x.lower().strip().decode('UTF-8') for x in cols]
                #line to features just makes easier to follow. 1st is lemma so strip that off.
                feats = cols[1:]
                new_word = True
                #print cols[2]
                #split features to pieces
                for feat in feats:
                    feat = feat.split(':')
                    if len(feat) <=1:
                        continue
                    #POS to dict and create inner dictionary for lemmas if POS doesn't exist
                    
                    
                    if root not in d:
                        try:
                            d[root] = {}
                        except:
                            print 'error on line {} for POS.'.format(i)
                            break
                    #Lemmas to dict if doen't exist create new dictionary for features
                    if cols[0] not in d[root]:
                        try:
                            d[root][cols[0]] = {}
                        except:
                            print 'error on line {} for lemma.'.format(i)
                            break
                    # total number of features per lemma
                    if feat[0] not in d[root][cols[0]]:
                        if new_word:
                            uusia += 1
                            new_word = False
                        try:
                            d[root][cols[0]][feat[0]] = int(feat[1])
                        except ValueError:
                            error_count += 1
                            continue
                    else:
                        if not new_word: 
                            olevia += 1
                        try:
                            d[root][cols[0]][feat[0]] += int(feat[1])
                        except ValueError:
                            error_count += 1
                            continue
            if rows < 0:
                if i%1000000 == 0:
                    print i
                continue
            elif i >= rows:
                print 'break {}'.format(i)
                break
        
        print 'New words {}, used words {} and errors {}.'.format(uusia, olevia, error_count)
        return d  

In [26]:
file_name = '../../../versions/2016-10-06/output.csv.gz'
dd = {}
#function call name of file and how many rows to read. easier to test in 1st place. set -1 for whole file.
%time dd = read_data(file_name, -1)

0
31000000
32000000
New words 1855261, used words 17127 and errors 76.
CPU times: user 1min 26s, sys: 548 ms, total: 1min 27s
Wall time: 1min 27s


In [27]:
print type(dd)
print [x + ' ' + str(len(dd[x])) for x in dd.keys() ]

<type 'dict'>
['ADJ 1844249']


In [32]:
dd['ADJ']['parka']

{u'case=abe': 1,
 u'case=abl': 40323,
 u'case=ade': 42,
 u'case=all': 7931,
 u'case=ela': 39,
 u'case=ess': 16597,
 u'case=gen': 108295,
 u'case=ill': 93976,
 u'case=ine': 18026,
 u'case=ins': 9,
 u'case=nom': 620124,
 u'case=par': 293011,
 u'case=tra': 7496,
 u'clitic=han': 10,
 u'clitic=kaan': 494,
 u'clitic=kin': 13951,
 u'clitic=ko': 4,
 u'degree=cmp': 1203989,
 u'degree=pos': 1366,
 u'degree=sup': 515,
 u'derivation=inen': 502,
 u'derivation=lainen': 11,
 u'number=plur': 130374,
 u'number=sing': 1075496,
 u'number[psor]=sing': 5,
 u'person[psor]=1': 1,
 u'person[psor]=2': 4,
 u'person[psor]=3': 332}

In [31]:
dd['ADJ'].vie

TypeError: unhashable type: 'list'

In [24]:
import lemmadict
lemmadict.write_to_csv(dd)

UnicodeEncodeError: 'ascii' codec can't encode character u'\xbb' in position 0: ordinal not in range(128)